* By: Illya Barziy
* Email: illyabarziy@gmail.com

## RiskEstimators class functions

This description is partially based on the User Guide of scikit-learn [available here](https://scikit-learn.org/stable/modules/covariance.html#robust-covariance).

## Introduction

Risk Eastimators class includes the implementations of functions for different ways to calculate and adjust Covaraince functions.

The following algorithms are now implemented:
- Minimum Covariance Determinant
- Maximum likelihood covariance estimator (Empirical covariance)
- Covariance estimator with shrinkage
  - Basic shrinkage
  - Ledoit-Wolf shrinkage
  - Oracle Approximating Shrinkage
- Semi-Covariance matrix
- Exponentially-weighted Covariance matrix
- De-noising covariance matrix
- Transforming covariance matrix to correlation matrix and back

This Notebook will describe the above algorithms as well as provide use cases and analysis of results.

## Minimum Covariance Determinant

The outliers are appearing in real data sets and seriously affect the Empirical covariance estimator and the Covariance estimators with shrinkage. For this reason, a robust covariance estimator is needed in order to discard/downweight the outliers in the data. 

The robust estimator presented in the package is the Minimum Covariance Determinant estimator, introduced by P.J. Rousseeuw.

The basic idea of the algorithm is to find a set of observations that are not outliers and compute their empirical covariance matrix, which is then rescaled to compensate for the performed selection of observations. 

Our function is a wrap around the sklearn's MinCovDet class, which uses FastMCD algorithm, developed by Rousseeuw and Van Driessen.

A detailed description of the algorithm is available in the paper by _Mia Hubert_ and _Michiel Debruyne_ __Minimum covariance determinant__ [available here](https://wis.kuleuven.be/stat/robust/papers/2010/wire-mcd.pdf)

### Examples of use

We can calculate the Minimum Covaraince Determinant estimator of covariance for a data set of stock prices and compare it to the simple covariance.

In [2]:
import mlfinlab as ml
import pandas as pd

In [3]:
# Getting the data
stock_prices = pd.read_csv('../Sample-Data/stock_prices.csv', parse_dates=True, index_col='Date')
stock_prices = stock_prices.dropna(axis=1)
stock_prices.head()

,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,49.273335,35.389999,106.639999,52.919998,78.220001,87.629997,37.939999,47.759998,41.299999,79.500000,...,42.090000,51.173328,55.983330,74.529999,67.309998,144.929993,94.379997,77.360001,101.400002,130.630005
2008-01-03,49.716667,35.290001,107.000000,53.119999,78.349998,87.809998,37.919998,48.060001,42.049999,80.440002,...,42.029999,51.293331,55.599998,74.800003,67.500000,144.860001,94.250000,77.459999,101.519997,130.740005
2008-01-04,48.223331,34.599998,106.970001,51.759998,76.570000,88.040001,36.990002,46.919998,40.779999,77.500000,...,42.349998,49.849998,54.536671,72.980003,65.769997,141.309998,94.269997,77.550003,101.650002,128.169998
2008-01-07,48.576668,34.630001,106.949997,51.439999,76.650002,88.199997,37.259998,47.060001,40.220001,77.199997,...,43.230000,50.416672,56.116669,72.949997,65.650002,141.190002,94.680000,77.570000,101.720001,128.059998
2008-01-08,48.200001,34.389999,107.029999,51.320000,76.220001,88.389999,36.970001,46.400002,39.599998,75.849998,...,43.240002,49.566669,55.326672,72.400002,65.360001,138.910004,94.570000,77.650002,101.739998,125.849998


In [4]:
# Leaving only 5 stocks in the dataset, so the differences between the 
# calculated covariance matrices would be easy to observe.
stock_prices = stock_prices.iloc[:, :5]
stock_prices.head()

,EEM,EWG,TIP,EWJ,EFA
Date,,,,,
2008-01-02,49.273335,35.389999,106.639999,52.919998,78.220001
2008-01-03,49.716667,35.290001,107.000000,53.119999,78.349998
2008-01-04,48.223331,34.599998,106.970001,51.759998,76.570000
2008-01-07,48.576668,34.630001,106.949997,51.439999,76.650002
2008-01-08,48.200001,34.389999,107.029999,51.320000,76.220001


In [5]:
# A class that has the Minimum Covariance Determinant estimator
risk_estimators = ml.portfolio_optimization.RiskEstimators()

# Finding the Minimum Covaraince Determinant estimator on price data and with set random seed to 0
min_cov_det = risk_estimators.minimum_covariance_determinant(stock_prices, price_data=True, random_state=0)

# For the simple covariance, we need to transform the stock prices to returns

# A class with function to calculate returns from prices
returns_estimation = ml.portfolio_optimization.ReturnsEstimation()

# Calcualting the data set of returns
stock_returns = returns_estimation.calculate_returns(stock_prices)

# Finding the simple covariance matrix from a series of returns
cov_matrix = stock_returns.cov()

# Transforming Minimum Covariance Determinant estimator from np.array to pd.DataFrame
min_cov_det = pd.DataFrame(min_cov_det, index=cov_matrix.index, columns=cov_matrix.columns)

print('The Minimum Covariance Determinant estimator is:')
min_cov_det

The Minimum Covariance Determinant estimator is:


,EEM,EWG,TIP,EWJ,EFA
EEM,0.000146,0.000112,-0.000005,0.000076,0.000102
EWG,0.000112,0.000154,-0.000007,0.000076,0.000114
TIP,-0.000005,-0.000007,0.000011,-0.000003,-0.000005
EWJ,0.000076,0.000076,-0.000003,0.000098,0.000077
EFA,0.000102,0.000114,-0.000005,0.000077,0.000100


In [6]:
print('The Simple Covariance is:')
cov_matrix

The Simple Covariance is:


,EEM,EWG,TIP,EWJ,EFA
EEM,0.000466,0.000350,-0.000017,0.000255,0.000324
EWG,0.000350,0.000372,-0.000015,0.000221,0.000303
TIP,-0.000017,-0.000015,0.000019,-0.000009,-0.000012
EWJ,0.000255,0.000221,-0.000009,0.000232,0.000218
EFA,0.000324,0.000303,-0.000012,0.000218,0.000278


From the results, the absolute values in the Minimum Covariance Determinant estimator are lower in comparison to the simple Covariance matrix, which means that the algorithm has eliminated some of the outliers in the data and the resulting covariance matrix estimator is a more robust one.